In [1]:
import json
import numpy as np
import pandas as pd
import os

C:\Users\wojtek\anaconda3\lib\site-packages\numpy\_distributor_init.py:30: UserWarning: loaded more than 1 DLL from .libs:
C:\Users\wojtek\anaconda3\lib\site-packages\numpy\.libs\libopenblas.EL2C6PLE4ZYW3ECEVIV3OXXGRN2NRFM2.gfortran-win_amd64.dll
C:\Users\wojtek\anaconda3\lib\site-packages\numpy\.libs\libopenblas.FB5AE2TYXYH2IJRDKGDGQ3XBKLKTF43H.gfortran-win_amd64.dll
  warnings.warn("loaded more than 1 DLL from .libs:"


In [35]:
os.listdir('data')

['dataset_crawler-google-places_2022-07-05_09-55-42-524.json',
 'dataset_crawler-google-places_2022-07-05_22-40-12-836.json',
 'dataset_crawler-google-places_2022-07-06_08-10-01-016.json',
 'dataset_crawler-google-places_2022-07-06_09-04-52-505.json']

In [37]:
data = []

for file in os.listdir('data'):
    with open('data/' + file, encoding='utf-8') as f:
        data.append(json.load(f))

In [38]:
len(data)

4

In [45]:
sum([len(x) for x in data])

418

In [49]:
sum([x['reviewsCount'] for data_pull in data for x in data_pull])

1490507

In [43]:
sum([len(x['reviews']) for data_pull in data for x in data_pull])

120853

In [42]:
len([r for data_pull in data for x in data_pull for r in x['reviews'] if r['text'] is not None])

75399

So the data comes from 418 restaurants and the sum of review counts is 149k, but there are actually only 120853 reviews scraped, 75399 with a text, probably due to per-location review limit I specified (max 500 per restaurants).

  
As for what data I want to extract, this would be, for each review, the text, the stars and the publishedAtDate
And the restaurant name as well. Not sure what else, but I can always revisit.

In [41]:
data_list = []
for data_pull in data:
    for rest in data_pull:
        for r in rest['reviews']:
            data_list.append({
                'name': rest['title'],
                'city': rest['city'],
                'category': rest['categoryName'],
                'stars': r['stars'],
                'publishedAt': r['publishedAtDate'],
                'text': r['text'] if r['text'] is not None else '',
                'reviewId': r['reviewId']
            })

len(data_list)

120853

In [44]:
df = pd.DataFrame(data_list)

In [24]:
df.head()

,name,category,stars,publishedAt,text,reviewId
0,Restauracja Zeppelin,Restaurant,5,2022-06-30T06:24:21.963Z,,ChdDSUhNMG9nS0VJQ0FnSURPM0lHMWp3RRAB
1,Restauracja Zeppelin,Restaurant,3,2020-09-11T18:41:45.985Z,(Translated by Google) It's hard for me to com...,ChdDSUhNMG9nS0VJQ0FnSURDME4yeDR3RRAB
2,Restauracja Zeppelin,Restaurant,4,2020-08-23T09:37:34.413Z,(Translated by Google) Place in a good locatio...,ChZDSUhNMG9nS0VJQ0FnSUNDN3REWFlnEAE
3,Restauracja Zeppelin,Restaurant,5,2020-08-20T16:42:43.746Z,"(Translated by Google) Delicious food, profess...",ChdDSUhNMG9nS0VJQ0FnSUNDMnFyOHNRRRAB
4,Restauracja Zeppelin,Restaurant,5,2020-03-25T21:20:03.338Z,(Translated by Google) With all the locality a...,ChZDSUhNMG9nS0VJQ0FnSUNjX2Ztc0FnEAE


In [50]:
df[df.publishedAt.str.contains('Z')].shape

(120853, 7)

In [51]:
df.publishedAt = df.publishedAt.apply(lambda x: pd.Timestamp(x.split('Z')[0]))

df.head()

,name,city,category,stars,publishedAt,text,reviewId
0,Restauracja Zeppelin,Gdańsk,Restaurant,5.0,2022-06-30 06:24:21.963,,ChdDSUhNMG9nS0VJQ0FnSURPM0lHMWp3RRAB
1,Restauracja Zeppelin,Gdańsk,Restaurant,3.0,2020-09-11 18:41:45.985,(Translated by Google) It's hard for me to com...,ChdDSUhNMG9nS0VJQ0FnSURDME4yeDR3RRAB
2,Restauracja Zeppelin,Gdańsk,Restaurant,4.0,2020-08-23 09:37:34.413,(Translated by Google) Place in a good locatio...,ChZDSUhNMG9nS0VJQ0FnSUNDN3REWFlnEAE
3,Restauracja Zeppelin,Gdańsk,Restaurant,5.0,2020-08-20 16:42:43.746,"(Translated by Google) Delicious food, profess...",ChdDSUhNMG9nS0VJQ0FnSUNDMnFyOHNRRRAB
4,Restauracja Zeppelin,Gdańsk,Restaurant,5.0,2020-03-25 21:20:03.338,(Translated by Google) With all the locality a...,ChZDSUhNMG9nS0VJQ0FnSUNjX2Ztc0FnEAE


In [59]:
df.drop_duplicates(subset=['reviewId']).name.nunique()

319

In [60]:
df.drop_duplicates(subset=['reviewId'], inplace=True)

In [61]:
df.to_csv('319_rest_revs.csv', index=False)
df.to_json('319_rest_revs.json')

In [62]:
df.category.value_counts()

Restaurant                    27705
Polish restaurant              6017
Pizza restaurant               6014
American restaurant            3925
European restaurant            3693
                              ...  
Small plates restaurant           6
Tourist information center        4
Non-profit organization           3
Takeaway                          2
Seafood wholesaler                1
Name: category, Length: 89, dtype: int64

In [63]:
df.name.value_counts()

Żabka                                    1259
Junior's Restaurant & Bakery             1000
McDonald's                               1000
Krowarzywa Vegan Burger                   537
Billy's American Restaurants Chmielna     508
                                         ... 
Planets Pride Polska Sp. z o.o.             1
Restauracja Qubus Gdańsk                    1
Meating Point Zwycięstwa 13a                1
Burgerland Meat & Vege                      1
Chilly Willy Gdańsk                         1
Name: name, Length: 319, dtype: int64

In [55]:
df.reviewId.nunique()

98464

In [65]:
df.city.value_counts()

Gdańsk              76186
Sopot                4225
Las Vegas            2716
Chicago              2342
New York             1260
Seattle              1088
Dayton                565
Ridgedale             561
Aventura              508
Colorado Springs      500
København K           500
Brooklyn              500
Omaha                 500
Asheville             500
Miami                 500
Paris                 500
Destin                500
Cherokee              500
Atlantic City         500
Lake Buena Vista      500
Vienna                480
Pebble Beach          449
Vitacura              358
Lynnwood              340
New Orleans           325
Gardone Riviera       320
Jensen Beach          275
Derby                 194
Orlando               167
Hyde Park             154
Raleigh               141
Atlantic Beach         88
Vero Beach             58
Chamonix               55
Southlake              49
Oxon Hill              46
Lexington               7
Des Moines              4
Philadelphia

In [68]:
df[df.city=='Gdańsk'].name.nunique(), df[df.city=='Sopot'].name.nunique()

(248, 14)

 All in all 248 restaurants in Gdansk and 14 in Sopot with over 80k reviews. Good start!

In [5]:
df = pd.read_csv('319_rest_revs.csv')
df.stars = df.stars.astype('Int64')
df.head()

,id,name,city,category,stars,published_at,review_text,review_id
0,0,Restauracja Zeppelin,Gdańsk,Restaurant,5,2022-06-30 06:24:21.963,NaN,ChdDSUhNMG9nS0VJQ0FnSURPM0lHMWp3RRAB
1,1,Restauracja Zeppelin,Gdańsk,Restaurant,3,2020-09-11 18:41:45.985,(Translated by Google) It's hard for me to com...,ChdDSUhNMG9nS0VJQ0FnSURDME4yeDR3RRAB
2,2,Restauracja Zeppelin,Gdańsk,Restaurant,4,2020-08-23 09:37:34.413,(Translated by Google) Place in a good locatio...,ChZDSUhNMG9nS0VJQ0FnSUNDN3REWFlnEAE
3,3,Restauracja Zeppelin,Gdańsk,Restaurant,5,2020-08-20 16:42:43.746,"(Translated by Google) Delicious food, profess...",ChdDSUhNMG9nS0VJQ0FnSUNDMnFyOHNRRRAB
4,4,Restauracja Zeppelin,Gdańsk,Restaurant,5,2020-03-25 21:20:03.338,(Translated by Google) With all the locality a...,ChZDSUhNMG9nS0VJQ0FnSUNjX2Ztc0FnEAE


In [4]:
df.reset_index(inplace=True)

In [6]:
df.rename(columns={'index': 'id'}, inplace=True)
df

,id,name,city,category,stars,published_at,review_text,review_id
0,0,Restauracja Zeppelin,Gdańsk,Restaurant,5.0,2022-06-30 06:24:21.963,NaN,ChdDSUhNMG9nS0VJQ0FnSURPM0lHMWp3RRAB
1,1,Restauracja Zeppelin,Gdańsk,Restaurant,3.0,2020-09-11 18:41:45.985,(Translated by Google) It's hard for me to com...,ChdDSUhNMG9nS0VJQ0FnSURDME4yeDR3RRAB
2,2,Restauracja Zeppelin,Gdańsk,Restaurant,4.0,2020-08-23 09:37:34.413,(Translated by Google) Place in a good locatio...,ChZDSUhNMG9nS0VJQ0FnSUNDN3REWFlnEAE
3,3,Restauracja Zeppelin,Gdańsk,Restaurant,5.0,2020-08-20 16:42:43.746,"(Translated by Google) Delicious food, profess...",ChdDSUhNMG9nS0VJQ0FnSUNDMnFyOHNRRRAB
4,4,Restauracja Zeppelin,Gdańsk,Restaurant,5.0,2020-03-25 21:20:03.338,(Translated by Google) With all the locality a...,ChZDSUhNMG9nS0VJQ0FnSUNjX2Ztc0FnEAE
...,...,...,...,...,...,...,...,...
98459,98459,Chinkalnia,Gdańsk,Georgian restaurant,5.0,2022-01-12 20:51:42.744,(Translated by Google) I'm glad that I discove...,ChZDSUhNMG9nS0VJQ0FnSUNtOV9TNk93EAE
98460,98460,Chinkalnia,Gdańsk,Georgian restaurant,5.0,2022-01-12 20:42:39.285,(Translated by Google) I was in a Georgian res...,ChZDSUhNMG9nS0VJQ0FnSUNtOTVTRUtREAE
98461,98461,Chinkalnia,Gdańsk,Georgian restaurant,5.0,2022-01-12 20:28:12.792,(Translated by Google) Very tasty. Lots of cho...,ChZDSUhNMG9nS0VJQ0FnSUNtOTdpN2NBEAE
98462,98462,Chinkalnia,Gdańsk,Georgian restaurant,5.0,2022-01-12 15:39:11.701,(Translated by Google) We were with my husband...,ChZDSUhNMG9nS0VJQ0FnSUNtdDdqM1R3EAE


In [6]:
df.to_csv('319_rest_revs.csv', index=False)

In [6]:
df.fillna({'review_text': ''}, inplace=True)
df.review_text = df.review_text.apply(lambda x: x.split('\n')[0])
df.head()

,id,name,city,category,stars,published_at,review_text,review_id
0,0,Restauracja Zeppelin,Gdańsk,Restaurant,5,2022-06-30 06:24:21.963,,ChdDSUhNMG9nS0VJQ0FnSURPM0lHMWp3RRAB
1,1,Restauracja Zeppelin,Gdańsk,Restaurant,3,2020-09-11 18:41:45.985,(Translated by Google) It's hard for me to com...,ChdDSUhNMG9nS0VJQ0FnSURDME4yeDR3RRAB
2,2,Restauracja Zeppelin,Gdańsk,Restaurant,4,2020-08-23 09:37:34.413,(Translated by Google) Place in a good locatio...,ChZDSUhNMG9nS0VJQ0FnSUNDN3REWFlnEAE
3,3,Restauracja Zeppelin,Gdańsk,Restaurant,5,2020-08-20 16:42:43.746,"(Translated by Google) Delicious food, profess...",ChdDSUhNMG9nS0VJQ0FnSUNDMnFyOHNRRRAB
4,4,Restauracja Zeppelin,Gdańsk,Restaurant,5,2020-03-25 21:20:03.338,(Translated by Google) With all the locality a...,ChZDSUhNMG9nS0VJQ0FnSUNjX2Ztc0FnEAE


In [7]:
df.review_text.values[:5]

array(['',
       "(Translated by Google) It's hard for me to comment .. we went to this restaurant for many years .. we held family parties and communion. Unfortunately, for the last few months the menu has been limited to items like .. um .. I would say bar-style .. burgers appeared, for example. The steaks are gone .. the fish .. today we just left the restaurant after a few minutes of studying the menu. We ate dinner elsewhere. But we came back for dessert in the form of ice cream .. actually .. everything would be ok .. if not for the bill .. the menu shows an ice cream dessert in the amount of PLN 12. However, there was 18 PLN on the bill. The nice waitress said that there was some error in the system. I hope the restaurant will return to its former shape. The ice cream from what I remember comes from the confectionery in Nowy Port and it was very good. But today hm .. I can't give more than 3 stars.",
       "(Translated by Google) Place in a good location - relatively quiet in 

In [9]:
df.to_csv('319_rest_revs_db.csv', index=False, header=False)